In [1]:
using Plots
gr()
#pyplot()

Plots.GRBackend()

In [2]:
#2016.07.17-i mérés számítása
module mérés
#Mért értékek
const t₀=54.741 #s
const t₁=56.621 #s <-4 ugrás+t₀
const t₂=59.419 #s <-10 ugrás+t₀
const m=70 #kg <-ugráló ember tömege
#Számított periódusidők
const Tₙ=(t₁-t₀)/4; #4 ugrás
const Tₜ=(t₂-t₀)/10; #10 ugrás
#Számított sajátkörfrekvenciák
const αₙ=2*pi/Tₙ #rad/s
const αₜ=2*pi/Tₜ #rad/s
#println(αₙ,"\t",αₜ)
#Számított merevségek
const sₙ=m*αₙ^2;
const sₜ=m*αₜ^2;
#println("T=$Tₙ [s]-hoz tartozó rugómerevség: s=$sₙ [N/m]")
println("T=$Tₜ [s]-hoz tartozó rugómerevség: s=$sₜ [N/m]")
export sₜ
end

T=0.4677999999999997 [s]-hoz tartozó rugómerevség: s=12628.080501916511 [N/m]


mérés

In [3]:
module hajotest
const m_hajo=55 #kg: hajó tömege
const merules=0.1 #m: a hajó merülése
const szelesseg=1.3 #m a hajó szélessége
const ρviz=1000 #kg/m³
const yst_hajo=0.1; #[m]
#A "hajó" merevsége
const s_hajo=9.81*m_hajo/yst_hajo;
show(s_hajo)
export s_hajo
export m_hajo
end

5395.5

hajotest

In [4]:
using mérés
using hajotest
 Δ=0.001; #számítások időbeli felbontása
 ϵ=0.000001; #0 ϵ sugarú környezetébe kell eljuttatni delta-t
#Modell paraméterei
const m1=70; #[kg]
const m2=55; #[kg]
const s1=sₜ;
const s2=s_hajo;
const g=9.81 #[m/s²]

#itt kell definiálni az elindító kezdeti feltételeket
const x0g=0.5; #[m]
const x10=-((m1+m2)*g/s2)-(m1*g/s1)-x0g #[m]
const x20=-((m1+m2)*g/s2) #[m]
const x1p0=0; #[m/s]
const x2p0=0; #[m/s]
const start_time=Float64(0);
    counter=100;

In [5]:
#sajátkör frekvencia és lengéskép számító függvény
function calculate_freq(m1,m2,s1,s2)
    M=[m1 0;0 m2];
    K=[s1 -s1;-s1 s1+s2];
    tmp1=eigvals(K',M);
    tmp2=eigvecs(K',M);
    alfa1=sqrt(tmp1[1])
    alfa2=sqrt(tmp1[2])
    V1=tmp2[:,1]
    V2=tmp2[:,2]
    return alfa1,alfa2,V1,V2
end

calculate_freq (generic function with 1 method)

In [6]:
#Mozgás első szakaszát leíró függvények (1db 2DOF rendszer)

const α1=calculate_freq(m1,m2,s1,s2)[1]; #[rad/s] 1. szakasz 1. sajátkörfrekvenciája
const α2=calculate_freq(m1,m2,s1,s2)[2]; #[rad/s] 1. szakasz 2. sajátkörfrekvenciája
const A1=calculate_freq(m1,m2,s1,s2)[3]; #1. szakasz első lengésképe
const A2=calculate_freq(m1,m2,s1,s2)[4]; #1. szakasz második lengésképe

2-element Array{Float64,1}:
 -0.0686458
  0.110383 

In [7]:
k1(x)=cos(α1*x)*A1[1]
k2(x)=sin(α1*x)*A1[1]
k3(x)=cos(α2*x)*A2[1]
k4(x)=sin(α2*x)*A2[1]

k5(x)=-1*sin(α1*x)*A1[1]*α1
k6(x)=cos(α1*x)*A1[1]*α1
k7(x)=-1*sin(α2*x)*A2[1]*α2
k8(x)=cos(α2*x)*A2[1]*α2

k9(x)=cos(α1*x)*A1[2]
k10(x)=sin(α1*x)*A1[2]
k11(x)=cos(α2*x)*A2[2]
k12(x)=sin(α2*x)*A2[2]

k13(x)=-1*sin(α1*x)*A1[2]*α1
k14(x)=cos(α1*x)*A1[2]*α1
k15(x)=-1*sin(α2*x)*A2[2]*α2
k16(x)=cos(α2*x)*A2[2]*α2

k16 (generic function with 1 method)

In [8]:
x1_2dof(cons,t)=cons[1]*k1(t)+cons[2]*k2(t)+cons[3]*k3(t)+cons[4]*k4(t)-((m1+m2)*g/s2)-m1*g/s1
x1p_2dof(cons,t)=cons[1]*k5(t)+cons[2]*k6(t)+cons[3]*k7(t)+cons[4]*k8(t)
x2_2dof(cons,t)=cons[1]*k9(t)+cons[2]*k10(t)+cons[3]*k11(t)+cons[4]*k12(t)-((m1+m2)*g/s2)
x2p_2dof(cons,t)=cons[1]*k13(t)+cons[2]*k14(t)+cons[3]*k15(t)+cons[4]*k16(t)

x2p_2dof (generic function with 1 method)

In [9]:
function initial_condition(allapot)#egy tömbbel tér vissza
    t=allapot[5];
    K=[k1(t) k2(t) k3(t) k4(t);k5(t) k6(t) k7(t) k8(t);k9(t) k10(t) k11(t) k12(t);k13(t) k14(t) k15(t) k16(t)];
    C=inv(K)*[allapot[1]+((m1+m2)*g/s2)+m1*g/s1;allapot[3];allapot[2]+((m1+m2)*g/s2);allapot[4]];
    return C # C[1]=c11 C[2]=c12 C[3]=c21 C[4]=c22 
    #initial_condition end
end

initial_condition (generic function with 1 method)

In [10]:
function calculate_2DOF(state)#paraméterek: kezdő állapot
    delta=-1;
    constant=initial_condition(state)
    t=state[5];
    i=state[6];
    while delta<0
        t+=Δ;
        i+=1;
        delta=x1_2dof(constant,t)-x2_2dof(constant,t);
        #println("$t időben $delta a különbség")
        if i>9223372036854775806
            println("ERROR: 9223372036854775806 ciklus után nincs gyök!")
              break;
            #if end
        end  
    end #while end
    
    time_begin=t-Δ;
    time_end=t;
    delta=1;
    lepeskoz=Δ;
    
    while delta>ϵ
    lepeskoz=lepeskoz/2;
    idotabla=collect(time_begin:lepeskoz:time_end);
    kulonbsegek=similar(idotabla,Float64)
    for (ijk,ido) in enumerate(idotabla)
        kulonbsegek[ijk]=abs(x1_2dof(constant,ido)-x2_2dof(constant,ido))
    end
    delta=minimum(kulonbsegek)   
    t=idotabla[indmin(kulonbsegek)]    
        
    end
    
    
    
    return [x1_2dof(constant,t) x2_2dof(constant,t) x1p_2dof(constant,t) x2p_2dof(constant,t) t i]
    #calculate_2DOF end 
end

calculate_2DOF (generic function with 1 method)

In [11]:
#Mozgás második szakaszát leíró függvények (2db 1DOF rendszer)
using hajotest
const α22=sqrt(s_hajo/m_hajo);
const yst=0.1;
#println(yst)
k17(x)=cos(α22*x)
k18(x)=sin(α22*x)
k19(x)=-sin(α22*x)*α22
k20(x)=cos(α22*x)*α22

k20 (generic function with 1 method)

In [12]:
function initial_condition(x1,x1p,t)#egy tömbbel tér vissza
    K=[k17(t) k18(t);k19(t) k20(t)];
    C=inv(K)*[x1+yst;x1p];
    return C # C[1]=c1 C[2]=C2 
end

initial_condition (generic function with 2 methods)

In [13]:
x1_1dof(x10,x1p0,Δt)=x10+x1p0*Δt-0.5*9.81*Δt^2
x1p_1dof(x1p0,Δt)=x1p0-g*Δt
x2_1dof(cons,t)=cons[1]*k17(t)+cons[2]*k18(t)-yst
x2p_1dof(cons,t)=cons[1]*k19(t)+cons[2]*k20(t)

x2p_1dof (generic function with 1 method)

In [14]:
function calculate_1DOF(state1)
    delta=1;  
    condition=initial_condition(state1[1],state1[3],state1[5]);    
    t=state1[5]
    i=state1[6];
    while delta>0
        t+=Δ;
        i+=1; 
        delta=x1_1dof(state1[1],state1[3],t-state1[5])-x2_1dof(condition,t);
        #println("$t időben $delta a különbség")
        if i>9223372036854775806
            println("ERROR: 9223372036854775806 ciklus után nincs gyök!")
            break;
            #if end
        end 
    end
    
    time_begin=t-Δ
    time_end=t;
    delta=1;
    lepeskoz=Δ
    
    while delta>ϵ
    lepeskoz=lepeskoz/2
    idotabla=collect(time_begin:lepeskoz:time_end)    
    kulonbsegek=similar(idotabla,Float64)
    for (ijk,ido) in enumerate(idotabla)
        kulonbsegek[ijk]=abs(x1_1dof(state1[1],state1[3],ido-state1[5])-x2_1dof(condition,ido))
    end
    delta=minimum(kulonbsegek)   
    t=idotabla[indmin(kulonbsegek)]
    end
    
    return [x1_1dof(state1[1],state1[3],t-state1[5]) x2_1dof(condition,t) x1p_1dof(state1[3],t-state1[5]) x2p_1dof(condition,t) t i]
end    

calculate_1DOF (generic function with 1 method)

In [15]:
#Szimuláció (és a hozzá szükséges globális változók):
#state változó: x1,x2,x1p,x2p,zerotime,times
function simulate(y1,y2,y1p,y2p,zero_time,times)
    state=[y1 y2 y1p y2p zero_time 1];
    sim_doc_num=zeros(Int64,1,times*2+1);
    sim_doc_num[1]=1;
    sim_doc_time=zeros(Float64,1,times*2+1);
    sim_doc_time[1]=0;
#szimuláció: kapcsolási pontok meghatározása
    for j=1:times
        state=calculate_2DOF(state)
        sim_doc_num[j*2]=state[6];
        sim_doc_time[j*2]=state[5];
        state=calculate_1DOF(state)
        sim_doc_num[1+j*2]=state[6];
        sim_doc_time[1+j*2]=state[5];
    end
    #szimulációs loop: állapotok mentése
    state_values=zeros(Float64,sim_doc_num[1+times*2],4);
    state_values[1,:]=[y1 y2 y1p y2p]
    for k in 1:times
        constants=initial_condition([state_values[sim_doc_num[2*k-1],1:4]; sim_doc_time[2*k-1]]);
        tim=sim_doc_time[2*k-1];
        for l in sim_doc_num[2*k-1]+1:sim_doc_num[2*k]
            tim+=Δ;
            state_values[l,:]=[x1_2dof(constants,tim) x2_2dof(constants,tim) x1p_2dof(constants,tim) x2p_2dof(constants,tim)]
        end
        cons=initial_condition(state_values[sim_doc_num[2*k],1],state_values[sim_doc_num[2*k],3],sim_doc_time[2*k])
        ti=sim_doc_time[2*k];
        for m in sim_doc_num[2*k]+1:sim_doc_num[2*k+1]
            ti+=Δ;
            state_values[m,:]=[x1_1dof(state_values[sim_doc_num[2*k],1],state_values[sim_doc_num[2*k],3],ti-sim_doc_time[2*k]) x2_1dof(cons,ti) x1p_1dof(state_values[sim_doc_num[2*k],3],ti-sim_doc_time[2*k]) x2p_1dof(cons, ti)]
        end
        end 
    return state_values,sim_doc_time
end

simulate (generic function with 1 method)

In [27]:
@time eredmeny=simulate(x10,x20,x1p0,x2p0,start_time,counter);

  0.600555 seconds (19.22 M allocations: 326.850 MB, 4.25% gc time)


In [17]:
#save("C:/Users/Lajos/Documents/Julia_hardfiles/eredmeny9.jld","eredmeny",eredmeny)

In [18]:
allapotok=eredmeny[1];
timeserie=eredmeny[2];
time_series=collect(start_time:Δ:timeserie[counter*2+1]);
    #=    for (j,dl) in enumerate(delta)
        delta[j]=allapotok[j,1]-allapotok[j,2]
    end
    
    
    plot(time_series,delta)
    mod_allapot=copy(allapotok);
        for (j,x1) in enumerate(mod_allapot[:,1])
        mod_allapot[j]+=1.7
        end
=#

39421-element Array{Float64,1}:
  0.0  
  0.001
  0.002
  0.003
  0.004
  0.005
  0.006
  0.007
  0.008
  0.009
  0.01 
  0.011
  0.012
  ⋮    
 39.409
 39.41 
 39.411
 39.412
 39.413
 39.414
 39.415
 39.416
 39.417
 39.418
 39.419
 39.42 

In [19]:
function showresults(tomb,idosor)
    for j in eachindex(tomb[:,1])
        t=idosor[j]
        x1=tomb[j,1]
        x2=tomb[j,2]
        x1p=tomb[j,3]
        x2p=tomb[j,4]
        println("t=$t\tx1=$x1\tx2=$x2\tx1p=$x1p\tx2p=$x2p")
    end
end

showresults (generic function with 1 method)

In [20]:
function allmultipliers(szam,min)
    i=2;
    kimenet=[1 ]
    while i<=szam/2
        if szam%i==0 && i>min
        push!(kimenet,i)
        end
       i+=1; 
    end
    return kimenet
end

allmultipliers (generic function with 1 method)

In [21]:
size(allapotok)

(39555,4)

In [22]:
#tombmeret=size(allapotok,1);
#allmultipliers(tombmeret,0);

In [23]:
pltranger=1;

In [24]:
plot(time_series,[allapotok[:,1],allapotok[:,2]])

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 -100 
 
 
 0 
 
 
 100 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 39.4926,193.518 39.5067,193.518 39.5208,193.518 39.5349,193.518 39.549,193.517 39.5632,193.517 39.5773,193.516 39.5914,193.516 39.6055,193.515 39.6196,193.514 
 39.6338,193.513 39.6479,193.512 39.662,193.511 39.6761,193.51 39.6902,193.509 39.7043,193.507 39.7185,193.506 39.7326,193.504 39.7467,193.503 39.7608,193.501 
 39.7749,193.499 39.7891,193.497 39.8032,193.495 39.8173,193.493 39.8314,193.491 39.8455,193.489 39.8597,193.487 39.8738,193.484 39.8879,193.482 39.902,193.479 
 39.9161,193.476 39.9303,193.474 39.9444,193.471 39.9585,193.468 39.9726,193.465 39.9867,193.462 40.0008,193.459 40.015,193.456 40.0291,193.453 40.0432,193.449 
 40.0573,193.446 40.0714,193.443 40.0856,193.439 40.0997,193.436 40.1138,193.432 40.1279,193.428 40.142,193.424 40.1562,193.421 40.1703,193.417 40.1844,193.413 
 40.1985,193.409 40.2126,193.405 40.2268,193.401 40.2409,193.397 40.255,193.393 40.2691,193.388 40.2832,193.384 40.2973,193.38 40.3115,193.376 40.3256,193.371 
 40.3397,193.367 40.3538,193.362 40.3679,193.358 40.3821,193.353 40.3962,193.349 40.4103,193.344 40.4244,193.34 40.4385,193.335 40.4527,193.331 40.4668,193.326 
 40.4809,193.321 40.495,193.317 40.5091,193.312 40.5232,193.307 40.5374,193.303 40.5515,193.298 40.5656,193.293 40.5797,193.288 40.5938,193.284 40.608,193.279 
 40.6221,193.274 40.6362,193.269 40.6503,193.265 40.6644,193.26 40.6786,193.255 40.6927,193.25 40.7068,193.246 40.7209,193.241 40.735,193.236 40.7492,193.232 
 40.7633,193.227 40.7774,193.222 40.7915,193.218 40.8056,193.213 40.8197,193.209 40.8339,193.204 40.848,193.199 40.8621,193.195 40.8762,193.19 40.8903,193.186 
 40.9045,193.181 40.9186,193.176 40.9327,193.172 40.9468,193.167 40.9609,193.163 40.9751,193.158 40.9892,193.154 41.0033,193.149 41.0174,193.145 41.0315,193.14 
 41.0457,193.136 41.0598,193.131 41.0739,193.127 41.088,193.123 41.1021,193.118 41.1162,193.114 41.1304,193.109 41.1445,193.105 41.1586,193.1 41.1727,193.096 
 41.1868,193.092 41.201,193.087 41.2151,193.083 41.2292,193.078 41.2433,193.074 41.2574,193.07 41.2716,193.065 41.2857,193.061 41.2998,193.056 41.3139,193.052 
 41.328,193.048 41.3422,193.043 41.3563,193.039 41.3704,193.035 41.3845,193.03 41.3986,193.026 41.4127,193.021 41.4269,193.017 41.441,193.013 41.4551,193.008 
 41.4692,193.004 41.4833,193 41.4975,192.995 41.5116,192.991 41.5257,192.987 41.5398,192.982 41.5539,192.978 41.5681,192.973 41.5822,192.969 41.5963,192.965 
 41.6104,192.96 41.6245,192.956 41.6386,192.952 41.6528,192.947 41.6669,192.943 41.681,192.938 41.6951,192.934 41.7092,192.929 41.7234,192.925 41.7375,192.921 
 41.7516,192.916 41.7657,192.912 41.7798,192.907 41.794,192.903 41.8081,192.898 41.8222,192.894 41.8363,192.889 41.8504,192.885 41.8646,192.881 41.8787,192.876 
 41.8928,192.872 41.9069,192.867 41.921,192.862 41.9351,192.858 41.9493,192.853 41.9634,192.849 41.9775,192.844 41.9916,192.84 42.0057,192.835 42.0199,192.831 
 42.034,192.826 42.0481,192.821 42.0622,192.817 42.0763,192.812 42.0905,192.807 42.1046,192.803 42.1187,192.798 42.1328,192.793 42.1469,192.789 42.1611,192.784 
 42.1752,192.779 42.1893,192.775 42.2034,192.77 42.2175,192.765 42.2316,192.76 42.2458,192.756 42.2599,192.751 42.274,192.746 42.2881,192.741 42.3022,192.736 
 42.3164,192.732 42.3305,192.727 42.3446,192.722 42.3587,192.717 42.3728,192.712 42.387,192.707 42.4011,192.702 42.4152,192.697 42.4293,192.693 42.4434,192.688 
 42.4576,192.683 42.4717,192.678 42.4858,192.673 42.4999,192.668 42.514,192.663 42.5281,192.658 42.5423,192.653 42.5564,192.648 42.5705,192.643 42.5846,192.638 
 42.5987,192.633 42.6129,192.628 42.627,192.623 42.6411,192.618 42.6552,192.613 42.6693,192.608 42.6835,192.603 42.6976,192.598 42.7117,192.592 42.7258,192.587 
 42.7399,192.582 42.754,192.57

In [25]:
#savefig("delta60eps90")